# Persiapan Analisis Sentimen

ambilkan data

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving DataCleanLabelPermen3021.csv to DataCleanLabelPermen3021.csv
Saving slang.csv to slang.csv


In [ ]:
!pip install PySastrawi

     |████████████████████████████████| 210 kB 5.0 MB/s 


In [ ]:
import pandas as pd
import re
import numpy as np
import nltk
import string
nltk.download('stopwords')
nltk.download('punkt')
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df = pd.read_csv('DataCleanLabelPermen3021.csv',sep=";", encoding='cp1252')
df

,date,tweet,retweet_count,favorite_count,username,follower,label
0,30/04/2022 12:29,saksi maupun pelapor akan mendapatkan pendampi...,0,0,himabiofmipaur,522,1
1,30/04/2022 12:29,Menurut Permendikbudristek RI No 30 Tahun 2021...,0,0,himabiofmipaur,522,1
2,30/04/2022 12:28,HIMABIO REPOST #Repost REPOST HOTLINE SATGAS P...,0,0,himabiofmipaur,522,3
3,30/04/2022 09:41,5 8 seperti mengesahkan Peraturan Rektor tenta...,0,0,BemFTUI,9458,1
4,30/04/2022 09:41,4 8 Pembentukan Pansel merupakan salah satu ke...,0,0,BemFTUI,9458,1
...,...,...,...,...,...,...,...
20685,26/10/2021 06:37,Alhamdulillah Permendikbud Pencegahan dan Pena...,1932,4342,Nisanurulhs,1828,1
20686,26/10/2021 06:09,Hari ini ada kabar gembira Permendikbud Penceg...,0,1,sugar2yourtea,38,1
20687,26/10/2021 05:59,Permendikbud tentang pencegahan KS dikit lg yuk,0,0,putriripuke,900,1
20688,25/10/2021 10:32,PERMENDIKBUDRISTEK NOMOR 30 TAHUN 2021 TENTANG...,0,0,ainamulyana1,195,1


In [ ]:
df['label'].value_counts()

1    12442
2     5115
3     3133
Name: label, dtype: int64

In [ ]:
labels = df['label']

sentimen_label = ['netral', 'positif', 'negatif']
sentiment_ordering = [3, 1, 2]
labels = labels.apply(lambda x: sentiment_ordering.index(x))
labels

0        1
1        1
2        0
3        1
4        1
        ..
20685    1
20686    1
20687    1
20688    1
20689    1
Name: label, Length: 20690, dtype: int64

In [ ]:
kamus_normalisasi = pd.read_csv("slang.csv",sep=";", encoding='cp1252')
normalisasi_dict = {}

for index, row in kamus_normalisasi.iterrows():
  if row[0] not in normalisasi_dict:
    normalisasi_dict[row[0]] = row[1]

case folding

In [ ]:
for indeks,tweets in enumerate(df['tweet']):
  # case folding
  tweets = tweets.lower()
  df['tweet'][indeks] = tweets

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


normalisasi tweet kata slang dan simbol penting

In [ ]:
sentimen = df['tweet']
tweetNormal = []
for indeks,tweets in enumerate(sentimen):
  # normalisasi
  # tweets = ' '.join(re.sub('2021|21'," duaribuduasatu",tweets).split())
  # tweets = ' '.join(re.sub('2022'," duaribuduadua",tweets).split())
  # tweets = ' '.join(re.sub('30'," tigapuluh",tweets).split())
  # tweets = ' '.join(re.sub('#'," htg",tweets).split())
  # tweets = ' '.join(re.sub('4|a{3,}',"a",tweets).split())
  # parnwk = ("wk{2,}|wkw{2,}|y{2,}|ww+|kwk+|\d{5,}|haha+|z{2,}|xix+|xixi+|ixi+|xx+|wq+|hehe+|waka(ka)+")
  # tweets = ' '.join(re.sub(parnwk," ",tweets).split())
  # tweets = ' '.join(re.sub('3|ee+',"e",tweets).split())
  # tweets = ' '.join(re.sub('h+',"h",tweets).split())
  # tweets = ' '.join(re.sub('1|i{3,}',"i",tweets).split())
  # tweets = ' '.join(re.sub('u{3,}',"u",tweets).split())
  # tweets = ' '.join(re.sub('k+',"k",tweets).split())
  # tweets = ' '.join(re.sub('s+',"s",tweets).split())
  # tweets = ' '.join(re.sub('o+',"o",tweets).split())
  # tweets = ' '.join(re.sub('r+',"r",tweets).split())
  # tweets = ' '.join(re.sub('g{3,}',"g",tweets).split())
  # tweets = ' '.join(re.sub("([^a-zA-Z])"," ",tweets).split())
  tweets = " ".join(normalisasi_dict.get(word, word) for word in tweets.split(' '))
  tweetNormal.append(tweets)

cekKataNormal = {'tweet': tweetNormal}
cekNormal = pd.DataFrame(cekKataNormal)
cekNormal['panjang'] = [len(kata) for kata in cekNormal['tweet']]
cekNormal

,tweet,panjang
0,saksi maupun pelapor akan mendapatkan pendampi...,203
1,menurut permendikbudristek ri no tigapuluh tah...,145
2,himabio repost #repost repost hotline satgas p...,76
3,lima 8 seperti mengesahkan peraturan rektor te...,223
4,empat 8 pembentukan pansel merupakan salah sat...,261
...,...,...
20685,alhamdulilah permendikbud pencegahan dan penan...,112
20686,hari ini ada kabar gembira permendikbud penceg...,211
20687,permendikbud tentang pencegahan kekerasan seks...,66
20688,permendikbudristek nomor tigapuluh tahun 2021 ...,128


Filtering Stopword

In [ ]:
stop_factory = StopWordRemoverFactory()
stopwordNLTK = stopwords.words('indonesian')
stopwordSastrawi = stop_factory.get_stop_words()
more_stopword = ['twitwar', 'repost', 'rt', 'are', 'deh','kah', 'htg']
filterStopword = set(stopwordSastrawi + stopwordNLTK + more_stopword)

tweetFilter = []
for kata in tweetNormal:
  removed = []
  tweet = []
  # tokenisasi
  tokens = kata.split(' ')
  # stopword
  for t in tokens:
    if t not in filterStopword:
      removed.append(t)
  for tweets in removed:
    tweet.append(tweets)
  tweets = ' '.join(tweet)
  tweetFilter.append(tweets)

cekFilter = {'tweet': tweetFilter}
cekFilterStop = pd.DataFrame(cekFilter)
cekFilterStop['panjang'] = [len(kata) for kata in cekFilterStop['tweet']]
cekFilterStop

,tweet,panjang
0,saksi pelapor pendampingan perlindungan grup s...,113
1,permendikbudristek ri no tigapuluh 2021 penceg...,109
2,himabio #repost hotline satgas ppks unri sahab...,57
3,8 mengesahkan peraturan rektor ppks ui kewajib...,130
4,8 pembentukan pansel salah kewajiban ui rangka...,174
...,...,...
20685,alhamdulilah permendikbud pencegahan penangana...,98
20686,kabar gembira permendikbud pencegahan penangan...,148
20687,permendikbud pencegahan kekerasan seksual ayo,45
20688,permendikbudristek nomor tigapuluh 2021 penceg...,100


Stemming Bahasa Indonesia

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

tweetStem = []
tweetJoin = []
for kata in tweetFilter:
  tweets = stemmer.stem(kata)
  # tweets = re.sub('htg',"#",tweets).split()
  tweetStem.append(tweets.split())
  tweetJoin.append(tweets)

data = {'tweet': tweetStem}
cekStemm = pd.DataFrame(data)
cekStemm['panjang'] = [len(kata) for kata in cekStemm['tweet']]
cekStemm

,tweet,panjang
0,"[saksi, lapor, damping, lindung, grup, satgas,...",13
1,"[permendikbudristek, ri, no, tigapuluh, 2021, ...",13
2,"[himabio, repost, hotline, satgas, ppks, unri,...",8
3,"[8, kesah, atur, rektor, ppks, ui, wajib, waji...",16
4,"[8, bentuk, pansel, salah, wajib, ui, rangka, ...",21
...,...,...
20685,"[alhamdulilah, permendikbud, cegah, tangan, ke...",9
20686,"[kabar, gembira, permendikbud, cegah, tangan, ...",15
20687,"[permendikbud, cegah, keras, seksual, ayo]",5
20688,"[permendikbudristek, nomor, tigapuluh, 2021, c...",10


# Text Preprocessing

In [ ]:
data = {'tweet': tweetStem, 'tweet join':tweetJoin, 'label':labels}
dataTweet = pd.DataFrame(data)
dataTweet

,tweet,tweet join,label
0,"[saksi, lapor, damping, lindung, grup, satgas,...",saksi lapor damping lindung grup satgas ppks t...,1
1,"[permendikbudristek, ri, no, tigapuluh, 2021, ...",permendikbudristek ri no tigapuluh 2021 cegah ...,1
2,"[himabio, repost, hotline, satgas, ppks, unri,...",himabio repost hotline satgas ppks unri sahaba...,0
3,"[8, kesah, atur, rektor, ppks, ui, wajib, waji...",8 kesah atur rektor ppks ui wajib wajib patut ...,1
4,"[8, bentuk, pansel, salah, wajib, ui, rangka, ...",8 bentuk pansel salah wajib ui rangka implemen...,1
...,...,...,...
20685,"[alhamdulilah, permendikbud, cegah, tangan, ke...",alhamdulilah permendikbud cegah tangan keras s...,1
20686,"[kabar, gembira, permendikbud, cegah, tangan, ...",kabar gembira permendikbud cegah tangan keras ...,1
20687,"[permendikbud, cegah, keras, seksual, ayo]",permendikbud cegah keras seksual ayo,1
20688,"[permendikbudristek, nomor, tigapuluh, 2021, c...",permendikbudristek nomor tigapuluh 2021 cegah ...,1


In [ ]:
# import ast

# def convert_text_list(texts):
#     texts = ast.literal_eval(str(texts))
#     return [text for text in texts]

# dataTweet["tweet_list"] = dataTweet["tweet"].apply(convert_text_list)
# print(dataTweet["tweet_list"][90])
# print("\ntype : ", type(dataTweet["tweet"][90]))

In [ ]:
def calc_TF(document):
    # Counts the number of times the word appears in review
    TF_dict = {}
    for term in document:
        if term in TF_dict:
            TF_dict[term] += 1
        else:
            TF_dict[term] = 1
    # Computes tf for each word
    for term in TF_dict:
        TF_dict[term] = TF_dict[term] / len(document)
    return TF_dict

dataTweet["TF_dict"] = dataTweet['tweet'].apply(calc_TF)
dataTweet["TF_dict"].head()

0    {'saksi': 0.07692307692307693, 'lapor': 0.0769...
1    {'permendikbudristek': 0.07692307692307693, 'r...
2    {'himabio': 0.125, 'repost': 0.125, 'hotline':...
3    {'8': 0.0625, 'kesah': 0.0625, 'atur': 0.0625,...
4    {'8': 0.047619047619047616, 'bentuk': 0.047619...
Name: TF_dict, dtype: object

In [ ]:
index = 12

print('%20s' % "term", "\t", "TF\n")
for key in dataTweet["TF_dict"][index]:
    print('%20s' % key, "\t", dataTweet["TF_dict"][index][key])

                term 	 TF

              undang 	 0.05263157894736842
                tpks 	 0.05263157894736842
              proses 	 0.05263157894736842
               legal 	 0.05263157894736842
               pakai 	 0.05263157894736842
  permendikbudristek 	 0.05263157894736842
                  no 	 0.05263157894736842
           tigapuluh 	 0.05263157894736842
               hukum 	 0.05263157894736842
              pidana 	 0.05263157894736842
                laku 	 0.05263157894736842
           mahasiswa 	 0.05263157894736842
              sanksi 	 0.05263157894736842
               tegur 	 0.05263157894736842
            skorsing 	 0.05263157894736842
                  do 	 0.05263157894736842
               dosen 	 0.05263157894736842
               henti 	 0.05263157894736842
              hormat 	 0.05263157894736842


In [ ]:
def calc_DF(tfDict):
    count_DF = {}
    for document in tfDict:
        for term in document:
            if term in count_DF:
                count_DF[term] += 1
            else:
                count_DF[term] = 1
    return count_DF

DF = calc_DF(dataTweet["TF_dict"])
DF
from nltk.probability import FreqDist
fdist = FreqDist(DF)

{'saksi': 58,
 'lapor': 285,
 'damping': 48,
 'lindung': 1088,
 'grup': 91,
 'satgas': 289,
 'ppks': 2942,
 'takut': 190,
 'suara': 318,
 'daretospeakup': 2,
 'togetherwecan': 2,
 'never': 6,
 'alone': 3,
 'permendikbudristek': 2528,
 'ri': 130,
 'no': 2474,
 'tigapuluh': 5898,
 '2021': 3589,
 'cegah': 1816,
 'tangan': 1250,
 'keras': 5109,
 'seksual': 5609,
 'lingkung': 1251,
 'guru': 1187,
 'pasal': 738,
 '10': 67,
 'himabio': 1,
 'repost': 5,
 'hotline': 12,
 'unri': 50,
 'sahabat': 33,
 'kreators': 3,
 '8': 29,
 'kesah': 102,
 'atur': 2378,
 'rektor': 149,
 'ui': 128,
 'wajib': 161,
 'patut': 44,
 'implementasi': 189,
 'wujud': 72,
 'aman': 330,
 'bebas': 1401,
 'bentuk': 407,
 'pansel': 22,
 'salah': 745,
 'rangka': 20,
 'permendikbud': 14038,
 'ristek': 1029,
 'sejati': 39,
 'milik': 156,
 'upaya': 188,
 'langkah': 212,
 'menteri': 763,
 'didik': 1207,
 'budaya': 272,
 'riset': 164,
 'teknologi': 156,
 'nomor': 858,
 'dukung': 2119,
 'universitas': 189,
 'negeri': 144,
 'yogyakar

In [ ]:
n_document = len(dataTweet)

def calc_IDF(__n_document, __DF):
    IDF_Dict = {}
    for term in __DF:
        IDF_Dict[term] = np.log(__n_document / (__DF[term] + 1))
    return IDF_Dict
  
#Stores the idf dictionary
IDF = calc_IDF(n_document, DF)
IDF

{'saksi': 5.859868326833869,
 'lapor': 4.281413959919736,
 'damping': 6.045585472628962,
 'lindung': 2.944390647806628,
 'grup': 5.415617193690548,
 'satgas': 4.267524847759069,
 'ppks': 1.9502210225061158,
 'takut': 4.685132342692959,
 'suara': 4.172214667954744,
 'daretospeakup': 8.838793482071479,
 'togetherwecan': 8.838793482071479,
 'never': 7.991495621684275,
 'alone': 8.551111409619699,
 'permendikbudristek': 2.1018265240696237,
 'ri': 5.0622084475384375,
 'no': 2.123410095736798,
 'tigapuluh': 1.2548676467365134,
 '2021': 1.7514982892572641,
 'cegah': 2.4324637023434175,
 'tangan': 2.8057072602726776,
 'keras': 1.3984510875418388,
 'seksual': 1.3050997722228468,
 'lingkung': 2.804908219079545,
 'guru': 2.8573792708169985,
 'pasal': 3.3321078497913867,
 '10': 5.717898065563482,
 'himabio': 9.244258590179644,
 'repost': 8.145646301511533,
 'hotline': 7.372456413278052,
 'unri': 6.005580138015263,
 'sahabat': 6.411045246123427,
 'kreators': 8.551111409619699,
 '8': 6.5362083890774

In [ ]:
#calc TF-IDF
def calc_TF_IDF(TF):
    TF_IDF_Dict = {}
    #For each word in the review, we multiply its tf and its idf.
    for key in TF:
        TF_IDF_Dict[key] = TF[key] * IDF[key]
    return TF_IDF_Dict

#Stores the TF-IDF Series
dataTweet["TF-IDF_dict"] = dataTweet["TF_dict"].apply(calc_TF_IDF)
dataTweet["TF-IDF_dict"]

0        {'saksi': 0.45075910206414377, 'lapor': 0.3293...
1        {'permendikbudristek': 0.16167896338997106, 'r...
2        {'himabio': 1.1555323237724555, 'repost': 1.01...
3        {'8': 0.4085130243173396, 'kesah': 0.331417298...
4        {'8': 0.31124801852749684, 'bentuk': 0.1869589...
                               ...                        
20685    {'alhamdulilah': 0.6694869739234937, 'permendi...
20686    {'kabar': 0.38218087675657486, 'gembira': 0.53...
20687    {'permendikbud': 0.0775622641535275, 'cegah': ...
20688    {'permendikbudristek': 0.21018265240696238, 'n...
20689    {'mahasiswa': 0.3193215224394728, 'desak': 0.4...
Name: TF-IDF_dict, Length: 20690, dtype: object

In [ ]:
index = 129

print('%20s' % "term", "\t", '%10s' % "TF", "\t", '%20s' % "TF-IDF\n")
for key in dataTweet["TF-IDF_dict"][index]:
    print('%20s' % key, "\t", dataTweet["TF_dict"][index][key] ,"\t" , dataTweet["TF-IDF_dict"][index][key])

                term 	         TF 	              TF-IDF

                laku 	 0.1 	 0.31011464934625216
               modus 	 0.1 	 0.729834844112433
              narasi 	 0.1 	 0.48016073336893267
                suka 	 0.2 	 0.6973870697190825
             lindung 	 0.1 	 0.29443906478066284
        permendikbud 	 0.1 	 0.03878113207676375
              korban 	 0.1 	 0.23450396422197928
               lemah 	 0.1 	 0.6411045246123428
            sengsara 	 0.1 	 0.8145646301511533


In [ ]:
sorted_DF = sorted(DF.items(), key=lambda kv: kv[1], reverse=True)
unique_term = [item[0] for item in sorted_DF]

def calc_TF_IDF_Vec(__TF_IDF_Dict):
    TF_IDF_vector = [0.0] * len(unique_term)
    for i, term in enumerate(unique_term):
        if term in __TF_IDF_Dict:
            TF_IDF_vector[i] = __TF_IDF_Dict[term]
    return TF_IDF_vector

dataTweet["TF_IDF_Vec"] = dataTweet["TF-IDF_dict"].apply(calc_TF_IDF_Vec)

print("print first row matrix TF_IDF_Vec Series\n")
print(unique_term)
print(dataTweet["TF_IDF_Vec"][0])
print("\nmatrix size : ", len(dataTweet["TF_IDF_Vec"][0]))

In [ ]:
for i in dataTweet['TF_IDF_Vec']:
  print(i)
  break

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
 modelGNB = GaussianNB()
# X_train, X_test, y_train, y_test = train_test_split(dataAryTweet, dataTweet['label'], test_size=0.2, random_state=0)

# gnbTrain = modelGNB.fit(X_train, y_train)
# y_preds = gnbTrain.predict(X_test)

# print(confusion_matrix(y_test,y_preds))
print(classification_report(y_test,y_preds))
print('nilai akurasinya adalah ',accuracy_score(y_test, y_preds))